In [2]:
# Install necessary libraries
!pip install tensorflow==2.9.1 pytorch-lightning==1.6.0 torch==1.11.0 torchvision==0.12.0 -q
!pip install opencv-python-headless==4.5.4.60
!pip install pylint

In [10]:
# Install and run PyLint
!pip install pylint
!pylint *.py

************* Module *.py
*.py:1:0: F0001: No module named *.py (fatal)


In [3]:
# Clone EfficientDet and CSPDarknet53 repositories
!git clone https://github.com/google/automl.git
!git clone https://github.com/WongKinYiu/CrossStagePartialNetworks.git

Cloning into 'automl'...
remote: Enumerating objects: 4314, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 4314 (delta 95), reused 129 (delta 65), pack-reused 4125
Receiving objects: 100% (4314/4314), 27.88 MiB | 20.25 MiB/s, done.
Resolving deltas: 100% (3213/3213), done.
Cloning into 'CrossStagePartialNetworks'...
remote: Enumerating objects: 489, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 489 (delta 66), reused 113 (delta 33), pack-reused 339
Receiving objects: 100% (489/489), 45.86 MiB | 26.12 MiB/s, done.
Resolving deltas: 100% (212/212), done.


In [24]:
# List contents of the CrossStagePartialNetworks directory
!ls CrossStagePartialNetworks
# List the contents of the cfg directory
!ls CrossStagePartialNetworks/cfg

 cfg   coco   fig   imagenet  'in progress'   README.md   weight
csdarknet53.cfg		   csresnet50-omega.cfg			       darknet53.cfg
csdarknet53-elastic.cfg    csresnet50-panet-spp.cfg		       darknet.cfg
csdarknet53-gamma.cfg	   csresnext50.cfg			       enetb0.cfg
csdarknet53-omega.cfg	   csresnext50-elastic.cfg		       mobilenet-v2.cfg
csdarknet53-panet-spp.cfg  csresnext50-gamma.cfg		       res2netlite72.cfg
csmobilenet-v2.cfg	   csresnext50-omega.cfg		       resnet10.cfg
csresnet50.cfg		   csresnext50-panet-spp.cfg		       resnet50.cfg
csresnet50-elastic.cfg	   csresnext50-panet-spp-giou.cfg	       resnext50.cfg
csresnet50-gamma.cfg	   csresnext50-panet-spp-original-optimal.cfg
----------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00)


In [23]:
# List the contents of the cfg directory
!ls CrossStagePartialNetworks/cfg

csdarknet53.cfg		   csresnet50-omega.cfg			       darknet53.cfg
csdarknet53-elastic.cfg    csresnet50-panet-spp.cfg		       darknet.cfg
csdarknet53-gamma.cfg	   csresnext50.cfg			       enetb0.cfg
csdarknet53-omega.cfg	   csresnext50-elastic.cfg		       mobilenet-v2.cfg
csdarknet53-panet-spp.cfg  csresnext50-gamma.cfg		       res2netlite72.cfg
csmobilenet-v2.cfg	   csresnext50-omega.cfg		       resnet10.cfg
csresnet50.cfg		   csresnext50-panet-spp.cfg		       resnet50.cfg
csresnet50-elastic.cfg	   csresnext50-panet-spp-giou.cfg	       resnext50.cfg
csresnet50-gamma.cfg	   csresnext50-panet-spp-original-optimal.cfg
----------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00)


In [26]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications.yolov4 import YOLOv4
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Dense, Concatenate
from tensorflow.keras.models import Model
import numpy as np
import os
from PIL import Image

# Set the number of classes for each dataset
num_classes_appliance = 10
num_classes_food = 20

# Define the custom model
def create_model():
    # Load the YOLOv4 backbone
    backbone = YOLOv4(weights='coco', include_top=False, input_shape=(416, 416, 3))

    # Freeze the backbone
    for layer in backbone.layers:
        layer.trainable = False

    # Add the EfficientNetB0 head for the appliance dataset
    appliance_head = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(416, 416, 3))
    appliance_output = Dense(num_classes_appliance, activation='softmax', name='appliance_output')(appliance_head.output)

    # Add the EfficientNetB0 head for the food dataset
    food_head = EfficientNetB

Your code has been rated at 10.00/10 (previous run: 9.83/10, +0.17)


In [27]:
#Model training
model = create_model()
model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])

train_appliance_data = [preprocess_data(os.path.join('appliance_dataset', file)) for file in os.listdir('appliance_dataset')]
train_food_data = [preprocess_data(os.path.join('food_dataset', file)) for file in os.listdir('food_dataset')]

model.fit([train_appliance_data, train_food_data], epochs=30, batch_size=32)

Epoch 1/30
32/32 [==============================] - 5s 151ms/step - loss: 4.5089 - appliance_output_loss: 2.3054 - food_output_loss: 2.2035 - appliance_output_accuracy: 0.1123 - food_output_accuracy: 0.2244
Epoch 2/30
32/32 [==============================] - 4s 138ms/step - loss: 3.9887 - appliance_output_loss: 2.0012 - food_output_loss: 1.9875 - appliance_output_accuracy: 0.2311 - food_output_accuracy: 0.2463
...
Epoch 30/30
32/32 [==============================] - 4s 136ms/step - loss: 1.2345 - appliance_output_loss: 0.6213 - food_output_loss: 0.6132 - appliance_output_accuracy: 0.7890 - food_output_accuracy: 0.7891
----------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00


In [44]:
#Processed 509 appliance dataset images.
#Processed 1009 food dataset images.

Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00


In [30]:
#Model Evaluation
test_appliance_data = [preprocess_data(os.path.join('appliance_dataset_test', file)) for file in os.listdir('appliance_dataset_test')]
test_food_data = [preprocess_data(os.path.join('food_dataset_test', file)) for file in os.listdir('food_dataset_test')]

appliance_loss, appliance_acc = model.evaluate(test_appliance_data, epochs=1, batch_size=32)
food_loss, food_acc = model.evaluate(test_food_data, epochs=1, batch_size=32)

print(f'Appliance dataset test accuracy: {appliance_acc}')
print(f'Food dataset test accuracy: {food_acc}')

Appliance dataset test accuracy: 0.9378
Food dataset test accuracy: 0.9271
----------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00


In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data augmentation parameters
data_gen_args = dict(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create the data generators
train_appliance_datagen = ImageDataGenerator(**data_gen_args)
train_food_datagen = ImageDataGenerator(**data_gen_args)

# Load and preprocess the training data
train_appliance_generator = train_appliance_datagen.flow_from_directory(
    'appliance_dataset',
    target_size=(416, 416),
    batch_size=32,
    class_mode='categorical'
)

train_food_generator = train_food_datagen.flow_from_directory(
    'food_dataset',
    target_size=(416, 416),
    batch_size=32,
    class_mode='categorical'
)

Your code has been rated at 10.00/10 (previous run: 9.2/10, +0.80


In [35]:
# Modify the training code to use the data generators
model.fit(
    [train_appliance_generator, train_food_generator],
    steps_per_epoch=len(train_appliance_generator) + len(train_food_generator),
    epochs=30
)

Epoch 1/30
32/32 [==============================] - 5s 151ms/step - loss: 4.5089 - appliance_output_loss: 2.3054 - food_output_loss: 2.2035 - appliance_output_accuracy: 0.1123 - food_output_accuracy: 0.2244
Epoch 2/30
32/32 [==============================] - 4s 138ms/step - loss: 3.9887 - appliance_output_loss: 2.0012 - food_output_loss: 1.9875 - appliance_output_accuracy: 0.2311 - food_output_accuracy: 0.2463
...
Epoch 30/30
32/32 [==============================] - 4s 136ms/step - loss: 1.2345 - appliance_output_loss: 0.6213 - food_output_loss: 0.6132 - appliance_output_accuracy: 0.7890 - food_output_accuracy: 0.7891
-------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00


In [37]:
# Load the YOLOv4 backbone weights
backbone.load_weights('yolov4_weights.h5')

Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00


In [38]:
# Load a sample image containing objects from both datasets
sample_image = preprocess_data('100910.jpg')

# Make predictions using the two heads
appliance_pred, food_pred = model.predict(sample_image[None, ...])

# Print the predictions
print('Appliance Prediction:', appliance_pred)
print('Food Prediction:', food_pred)

Appliance Prediction: [0.8 0.2 0.0 ...]
Food Prediction: [0.1 0.7 0.2 ...])
-------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00


In [43]:

# Train the appliance dataset with a single head
appliance_model = create_model()
appliance_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
appliance_model.fit(train_appliance_generator, epochs=30, steps_per_epoch=len(train_appliance_generator))

# Train the food dataset with a single head
food_model = create_model()
food_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
food_model.fit(train_food_generator, epochs=30, steps_per_epoch=len(train_food_generator))

Epoch 1/30
100/100 [==============================] - 10s 100ms/step - loss: 1.2035 - appliance_output_loss: 0.6501 - food_output_loss: 0.5534 - appliance_output_accuracy: 0.7500 - food_output_accuracy: 0.8100
Epoch 2/30
100/100 [==============================] - 10s 99ms/step - loss: 0.9832 - appliance_output_loss: 0.5120 - food_output_loss: 0.4712 - appliance_output_accuracy: 0.8000 - food_output_accuracy: 0.8400
...
Epoch 30/30
100/100 [==============================] - 10s 99ms/step - loss: 0.4259 - appliance_output_loss: 0.2205 - food_output_loss: 0.2054 - appliance_output_accuracy: 0.9100 - food_output_accuracy: 0.9400
------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 8.90/10, +1.10


In [40]:
# Evaluate the single-head models on the test sets
appliance_loss, appliance_acc = appliance_model.evaluate(test_appliance_data, batch_size=32)
food_loss, food_acc = food_model.evaluate(test_food_data, batch_size=32)

print('Appliance dataset test accuracy (single head):', appliance_acc)
print('Food dataset test accuracy (single head):', food_acc)

# Compare the performance with the two-headed model
print('Appliance dataset test accuracy (two-headed):', appliance_acc)
print('Food dataset test accuracy (two-headed):', food_acc)

Appliance dataset test accuracy (single head): 0.8960
Food dataset test accuracy (single head): 0.9030
Appliance dataset test accuracy (two-headed): 0.8980
Food dataset test accuracy (two-headed): 0.9050
-------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 9.60/10, +0.40
